In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.optim import Adam
from torchvision.transforms import transforms, Compose, ToTensor, Lambda
from torchvision.datasets.mnist import MNIST, FashionMNIST
from torch.utils.data import DataLoader, random_split
from tools.models import *
from tools.data_utils import *
import matplotlib.pyplot as plt
import numpy as np
import random
import imageio
from argparse import ArgumentParser
import einops
from tools.plot_utils import show_images, show_forward, generate_new_images
from tools.models import Autoencoder
from torch.utils.data import Subset

# Setting reproducibility
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Definitions
STORE_PATH_MNIST = f"./history/ddpm_model_mnist.pt"
STORE_PATH_SYNTH = f"./history/ddpm_model_synth.pt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
unet = UNet(4, shape=100)
print(unet)

UNet(
  (time_embed): Embedding(1000, 100)
  (te1): Sequential(
    (0): Linear(in_features=100, out_features=4, bias=True)
    (1): SiLU()
    (2): Linear(in_features=4, out_features=4, bias=True)
  )
  (b1): Sequential(
    (0): MyBlock(
      (ln): LayerNorm((4, 100, 100), eps=1e-05, elementwise_affine=True)
      (conv1): Conv2d(4, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (activation): SiLU()
    )
    (1): MyBlock(
      (ln): LayerNorm((10, 100, 100), eps=1e-05, elementwise_affine=True)
      (conv1): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (activation): SiLU()
    )
    (2): MyBlock(
      (ln): LayerNorm((10, 100, 100), eps=1e-05, elementwise_affine=True)
      (conv1): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(10, 10, ker

In [6]:
# Set the seed for reproducibility
torch.manual_seed(42)

# Define the input dimensions
batch_size = 2
channels = 4
height = 100
width = 100

# Generate random inputs for testing
input_data = torch.randn(batch_size, channels, height, width)
n_steps = 1000
time_steps = torch.randint(0, n_steps, (batch_size,))
output = unet(input_data, time_steps)

In [9]:
input_data[0]

tensor([[[ 1.9269,  1.4873,  0.9007,  ...,  0.4880,  0.7846,  0.0286],
         [ 0.6408,  0.5832,  1.0669,  ...,  0.3581,  0.4788,  1.3537],
         [ 0.5261,  2.1120, -0.5208,  ...,  0.2539,  0.9364,  0.7122],
         ...,
         [-0.2736,  1.4906,  0.8558,  ...,  0.7944,  0.0610,  0.0932],
         [-0.4450, -0.1099, -0.5998,  ..., -0.1095,  1.8557, -2.2152],
         [ 0.9302,  1.6466,  1.3799,  ...,  0.8227, -0.8546,  0.7941]],

        [[-1.1723,  0.4068, -0.6165,  ..., -0.0192,  0.0207, -0.2878],
         [-0.5640, -0.1440, -0.5088,  ..., -1.0222,  0.5673, -1.1977],
         [-0.0802,  0.5409, -0.4572,  ..., -0.0517, -0.4695, -0.3873],
         ...,
         [ 0.8734,  0.8646,  0.0357,  ..., -0.3075,  1.3650,  0.5020],
         [-0.8316,  0.6496,  0.0167,  ...,  0.6172, -1.9147, -0.9924],
         [ 1.4250, -0.6707,  0.3655,  ...,  0.6578, -0.6725, -1.2849]],

        [[ 1.4459,  0.4772, -0.5228,  ...,  0.1008,  0.6087,  1.0949],
         [-0.6739,  1.1621,  1.1202,  ..., -1

In [8]:
output[0]

tensor([[[ 2.1395e-02,  2.0801e-02,  1.8702e-02,  ...,  1.6494e-02,
           1.6122e-02,  2.2055e-02],
         [ 2.8393e-02,  3.2001e-02,  3.0398e-02,  ...,  2.7426e-02,
           2.7457e-02,  2.9665e-02],
         [ 2.6516e-02,  3.2388e-02,  2.8632e-02,  ...,  2.6351e-02,
           2.7034e-02,  2.7365e-02],
         ...,
         [ 2.5700e-02,  3.0168e-02,  2.6019e-02,  ...,  2.5457e-02,
           2.4742e-02,  2.7002e-02],
         [ 2.7601e-02,  3.3288e-02,  2.9509e-02,  ...,  2.8398e-02,
           2.7227e-02,  2.7824e-02],
         [ 3.2938e-02,  3.1317e-02,  2.9038e-02,  ...,  2.9103e-02,
           2.8610e-02,  2.9180e-02]],

        [[ 1.0974e-01,  1.0383e-01,  1.0372e-01,  ...,  1.0300e-01,
           1.0125e-01,  1.0323e-01],
         [ 1.0474e-01,  9.7586e-02,  9.7670e-02,  ...,  9.8008e-02,
           9.3648e-02,  9.7954e-02],
         [ 1.0564e-01,  9.8228e-02,  9.8884e-02,  ...,  9.5183e-02,
           9.3588e-02,  9.7373e-02],
         ...,
         [ 1.0610e-01,  9

In [ ]:
train_losses = []
test_losses = []


def training_loop(dataset, ddpm, loader, n_epochs, optim, device, display=False, store_path="ddpm_model.pt"):
    mse = nn.MSELoss()
    best_loss = float("inf")
    n_steps = ddpm.n_steps
    for epoch in range(num_epochs):
        train_loss = 0
        for data in loader:
            # Loading data
            if dataset =="MNIST":
                x0 = data[0].to(device)
            else:
                x0 = data["image"].float().to(device)
            n = len(x0)

            # Picking some noise for each of the images in the batch, a timestep and the respective alpha_bars
            eta = torch.randn_like(x0).to(device)
            t = torch.randint(0, n_steps, (n,)).to(device)

            # Computing the noisy image based on x0 and the time-step (forward process)
            noisy_imgs = ddpm(x0, t, eta)

            # Getting model estimation of noise based on the images and the time-step
            eta_theta, _ = ddpm.backward(noisy_imgs, t.reshape(n, -1))

            # Optimizing the MSE between the noise plugged and the predicted noise
            loss = mse(eta_theta, eta)
            optim.zero_grad()
            loss.backward()
            optim.step()

            train_loss += loss.item() * len(x0) / len(loader.dataset)

        # Display images generated at this epoch
        if display:
            show_images(generate_new_images(ddpm, device=device), f"Images generated at epoch {epoch + 1}")

        log_string = f"Loss at epoch {epoch + 1}: {train_loss:.3f}"
    
        # Storing the model
        if best_loss > train_loss:
            best_loss = train_loss
            torch.save(ddpm.state_dict(), store_path)
            log_string += " --> Best model ever (stored)"

        print(log_string)

In [ ]:
# Hyperparameters
dataset = "Building"
store_path = f"./history/ddpm_model_{dataset}.pt"
batch_size = 128
learning_rate = 0.001
num_epochs = 20
input_channel = 1 if dataset == "PinMNIST" else 4

# Loading the data (converting each image into a tensor and normalizing between [-1, 1])
if dataset == "Building":
    resize = Resize100
else:
    resize = Resize
    
if dataset == "MNIST":
    ds_fn = MNIST
    dataset_fn = ds_fn("./datasets", download=True, train=True, transform=transform)
    train_dataloader = DataLoader(dataset_fn, batch_size, shuffle=True)

    if dataset == "Building":
        resize = Resize100
    else:
        resize = Resize
elif dataset == "Synthetic":
    # Use enough images to get great generation
    data_folder = "./data/Synthetic/10images_28by28pixels_4_distanced_grid_pins_4seed"
    dataset_fn = PinDataset(csv_file=f"{data_folder}/pins.csv",
                                  root_dir=f"{data_folder}/images/",
                                  transform=transforms.Compose([ToTensor(), resize()]))
    train_dataloader = DataLoader(dataset_fn, batch_size, shuffle=True)
    
elif dataset == "Building":
    transformed_dataset = PinDataset(csv_file=f"./data/{dataset}/pins_full.csv",
                             root_dir=f"./data/{dataset}/PS-RGBNIR/",
                             transform=Compose([ToTensor(), resize(), Lambda()]))
            
if os.path.exists(f"./data/{dataset}/train_indices.npy"):
    train_indices = np.load(f'./data/{dataset}/train_indices.npy')
    val_indices = np.load(f'./data/{dataset}/val_indices.npy')
    test_indices = np.load(f'./data/{dataset}/test_indices.npy')
    train_indices = np.concatenate((train_indices, np.arange(1000, 1697))) # add all un-used images for DDPM training
    # Use the indices to create new datasets
    train_dataset = Subset(transformed_dataset, train_indices)
    val_dataset = Subset(transformed_dataset, val_indices)
    test_dataset = Subset(transformed_dataset, test_indices)
else:
    # Split the dataset into train, validation, and test sets
    train_dataset, val_dataset, test_dataset = random_split(
        transformed_dataset, [train_size, val_size, test_size]
    )
    np.save(f'./data/{dataset}/train_indices.npy', train_dataset.indices)
    np.save(f'./data/{dataset}/val_indices.npy', val_dataset.indices)
    np.save(f'./data/{dataset}/test_indices.npy', test_dataset.indices)

# Create your DataLoader with the custom_collate_fn
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)

# Initialize the autoencoder
n_steps, min_beta, max_beta = 1000, 10 ** -4, 0.02  # Originally used by the authors
if dataset == "Building":
    model = DDPM(UNet(input_channel, shape=100), n_steps=n_steps, min_beta=min_beta, max_beta=max_beta, device=device)
else:
    model = DDPM(UNet(input_channel), n_steps=n_steps, min_beta=min_beta, max_beta=max_beta, device=device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
training_loop(dataset, model, train_loader, num_epochs, optimizer, device, store_path=store_path)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

count = 0
for batch in train_loader:
    images = batch['image'].to(device) # get RGB instead of RGBA
    pins = batch['pins']
    outputs = batch['outputs']
    print(count, len(images))
    count+=1

In [ ]:
# Loading the trained model
store_path = f"./history/ddpm_model_{dataset}.pt"
best_model = DDPM(UNet(), n_steps=n_steps, device=device)
best_model.load_state_dict(torch.load(store_path, map_location=device))
best_model.eval()
print("Model loaded")

print("Generating new images")
generated = generate_new_images(
        best_model,
        n_samples=100,
        device=device,
        gif_name="mnist.gif"
    )
show_images(generated, "Final result")

In [ ]:
show_forward(model, train_dataloader, device)

In [ ]:
# Visual evaluation on the test set

# change for dataset
num_kernels_encoder = [16, 8]
num_kernels_decoder = [16]
input_channel = 1

test_dataloader = DataLoader(dataset, batch_size, shuffle=True)

with torch.no_grad():
    for data in test_dataloader:
        inputs, _ = data
        inputs = inputs.view(inputs.size(0), -1)
        
        # Forward pass through the autoencoder
        autoencoder = Autoencoder(num_kernels_encoder, num_kernels_decoder, input_channel=input_channel).to(device)
        outputs = autoencoder(inputs)
        
        # Reshape the outputs to the original image shape (28x28)
        outputs = outputs.view(outputs.size(0), 1, 28, 28)
        
        # Convert Tensors to NumPy arrays for visualization
        inputs = inputs.view(inputs.size(0), 1, 28, 28).numpy()
        outputs = outputs.numpy()
        
        # Plot the input and output images
        for i in range(inputs.shape[0]):
            plt.figure(figsize=(8, 4))
            plt.subplot(1, 2, 1)
            plt.title('Input Image')
            plt.imshow(np.squeeze(inputs[i]), cmap='gray')
            plt.axis('off')
            
            plt.subplot(1, 2, 2)
            plt.title('Output Image')
            plt.imshow(np.squeeze(outputs[i]), cmap='gray')
            plt.axis('off')
            plt.show()